### 责任链模式 

开发一个应用时，多数时候我们都能预先知道哪个方法能处理某个特定请求。然而，情况并
非总是如此。例如，想想任意一种广播计算机网络，例如最早的以太网实现（请参考网页
［t.cn/RqrTp0Y］）。在广播计算机网络中，会将所有请求发送给所有节点（简单起见，不考虑广播
域） ，但仅对所发送请求感兴趣的节点会处理请求。加入广播网络的所有计算机使用一种常见的
媒介相互连接。 

如果一个节点对某个请求不感兴趣或者不知道如何处理这个请求，可以执行以下两个操作。    
 忽略这个请求，什么都不做    
 将请求转发给下一个节点      
节点对一个请求的反应方式是实现的细节。然而，我们可以使用广播计算机网络的类比来
理解责任链模式是什么。责任链（Chain of Responsibility）模式用于让多个对象来处理单个请求
时，或者用于预先不知道应该由哪个对象（来自某个对象链）来处理某个特定请求时。其原则
如下所示。    
(1) 存在一个对象链（链表、树或任何其他便捷的数据结构）。   
(2) 我们一开始将请求发送给链中的第一个对象。   
(3) 对象决定其是否要处理该请求。   
(4) 对象将请求转发给下一个对象。   
(5) 重复该过程，直到到达链尾。   

应用级别，不用讨论光缆和网络节点，而是可以专注于对象以及请求的流程.
注意，客户端代码仅知道第一个处理元素，而非拥有对所有处理元素的引用；并且每个处理
元素仅知道其直接的下一个邻居（称为后继），而不知道所有其他处理元素。这通常是一种单向
关系，用编程术语来说是一个单向链表，与之相反的是双向链表。单向链表不允许双向地遍历元
素，双向链表则是允许的。这种链式组织方式大有用处：可以解耦发送方（客户端）和接收方（处
理元素）。 

现实生活的例子 
ATM机以及及一般而言用于接收/返回钞票或硬币的任意类型机器（比如，零食自动贩卖机）
都使用了责任链模式。机器上总会有一个放置各种钞票的槽口,钞票放入之后，会被传递到恰当的容器。钞票返回时，则是从恰当的容器中获取（请参考网
页［t.cn/RqrTYuB］和网页［t.cn/RqrTnts］）。我们可以把这个槽口视为共享通信媒介，不同的容
器则是处理元素。结果包含来自一个或多个容器的现金。

### 应用案例 
通过使用责任链模式，我们能让许多不同对象来处理一个特定请求。在我们预先不知道应该
由哪个对象来处理某个请求时，这是有用的。其中一个例子是采购系统。在采购系统中，有许多
核准权限。某个核准权限可能可以核准在一定额度之内的订单，假设为100美元。如果订单超过了
100美元，则会将订单发送给链中的下一个核准权限，比如能够核准在200美元以下的订单，等等。  
另一个责任链可以派上用场的场景是，在我们知道可能会有多个对象都需要对同一个请求进
行处理之时。这在基于事件的编程中是常有的事情。单个事件，比如一次鼠标左击，可被多个事
件监听者捕获。 
不过应该注意，如果所有请求都能被单个处理程序处理，责任链就没那么有用了，除非确实
不知道会是哪个程序处理请求。这一模式的价值在于解耦。客户端与所有处理程序（一个处理程
序与所有其他处理程序之间也是如此）之间不再是多对多关系，客户端仅需要知道如何与链的起
始节点（标头）进行通信。 

### 实现 
使用Python实现责任链模式有许多种方式，但是我最喜欢的实现是Vespe Savikko所提出的
（请参考网页［t.cn/RqruSj1］）。Vespe的实现以地道的Python风格使用动态分发来处理请求（请参
考网页［t.cn/RqruWFp］）。 
我们以Ve spe的实现为参考实现一个简单的事件系统.Event类描述一个事件。为了让它简单一点，在我们的案例中一个事件只有一个name属性

In [ ]:
class Event:

    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name


Widget类是应用的核心类。 UML图中展示的parent聚合关系表明每个控件都有一个到父对
象的引用。按照约定，我们假定父对象是一个Widget实例。然而，注意，根据继承的规则，任
何Widget子类的实例（例如，MsgText的实例）也是Widget实例。parent的默认值为None。 

In [ ]:
class Widget:

    def __init__(self, parent=None):
        self.parent = parent

    def handle(self, event):
        handler = 'handle_{}'.format(event)
        if hasattr(self, handler):
            method = getattr(self, handler)
            method(event)
        elif self.parent:
            self.parent.handle(event)
        elif hasattr(self, 'handle_default'):
            self.handle_default(event)

handle()方法使用动态分发，通过hasattr()和getattr()决定一个特定请求（event）
应该由谁来处理。如果被请求处理事件的控件并不支持该事件，则有两种回退机制。如果控件有
parent，则执行parent的handle()方法。如果控件没有parent，但有handle_default()方
法，则执行handle_default()。    
此时，你可能已明白为什么UML类图中Widget与Event类仅是关联关系而已（不是聚合或
组合关系）。关联关系用于表明Widget类知道Event类，但对其没有任何严格的引用，因为事件
仅需要作为参数传递给handle()。 

MainWindow、MsgText和SendDialog是具有不同行为的控件。我们并不期望这三个控件
都能处理相同的事件，即使它们能处理相同事件，表现出来也可能是不同的。MainWindow仅能
处理close和default事件。 

In [1]:
class Event:

    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name


class Widget:

    def __init__(self, parent=None):
        self.parent = parent

    def handle(self, event):
        handler = 'handle_{}'.format(event)
        if hasattr(self, handler):
            method = getattr(self, handler)
            method(event)
        elif self.parent:
            self.parent.handle(event)
        elif hasattr(self, 'handle_default'):
            self.handle_default(event)


class MainWindow(Widget):

    def handle_close(self, event):
        print('MainWindow: {}'.format(event))

    def handle_default(self, event):
        print('MainWindow Default: {}'.format(event))


class SendDialog(Widget):

    def handle_paint(self, event):
        print('SendDialog: {}'.format(event))


class MsgText(Widget):

    def handle_down(self, event):
        print('MsgText: {}'.format(event))


def main():
    mw = MainWindow()
    sd = SendDialog(mw)
    msg = MsgText(sd)

    for e in ('down', 'paint', 'unhandled', 'close'):
        evt = Event(e)
        print('\nSending event -{}- to MainWindow'.format(evt))
        mw.handle(evt)
        print('Sending event -{}- to SendDialog'.format(evt))
        sd.handle(evt)
        print('Sending event -{}- to MsgText'.format(evt))
        msg.handle(evt)

main()函数展示如何创建一些控件和事件，以及控件如何对那些事件作出反应。所有事件
都会被发送给所有控件。注意其中每个控件的父子关系。sd对象（SendDialog的一个实例）的
父对象是mw（MainWindow的一个实例）。然而，并不是所有对象都需要一个MainWindow实例的
父对象。例如，msg对象（MsgText的一个实例）是以sd作为父对象。 

In [2]:
main()


Sending event -down- to MainWindow
MainWindow Default: down
Sending event -down- to SendDialog
MainWindow Default: down
Sending event -down- to MsgText
MsgText: down

Sending event -paint- to MainWindow
MainWindow Default: paint
Sending event -paint- to SendDialog
SendDialog: paint
Sending event -paint- to MsgText
SendDialog: paint

Sending event -unhandled- to MainWindow
MainWindow Default: unhandled
Sending event -unhandled- to SendDialog
MainWindow Default: unhandled
Sending event -unhandled- to MsgText
MainWindow Default: unhandled

Sending event -close- to MainWindow
MainWindow: close
Sending event -close- to SendDialog
MainWindow: close
Sending event -close- to MsgText
MainWindow: close
